###Potential Interview Questions

- What is repartitioning in spark ? 
- What is coalesce in spark ?
- Which one will you choose and why ?
- Repartitioning vs coalesce ?  


###### What is repartitioning in Spark?

```Repartitioning is the process of redistributing data across a cluster to achieve a more optimal data partitioning.```

* In Spark, the repartition method redistributes data across a specified number of partitions. It can be used to either<br>
  increase or decrease the number of partitions.
* The repartition() function is used to increase or decrease the number of partitions in a DataFrame.
*  This method performs a full shuffle of data across all the nodes. It creates partitions of more or less equal in size.<br> 
   This is a costly operation given that it involves data movement all over the network.

###### What is coalesce in Spark?
```In Spark, coalesce is a method that reduces the number of partitions in a DataFrame.```

* It does this by shuffling the data using a Hash Partitioner and adjusting it into existing partitions.<br>
  This means that it can only decrease the number of partitions.
* It avoids a full shuffle.

###### Here are some screenshots from the instructor of this lecture video

<img src="https://drive.google.com/uc?id=1zD6XUnVGIluVZMpUoHbzbl0omFnBGg4c" alt="drawing" style="width:500px;"/><br>

<img src="https://drive.google.com/uc?id=1dGLprdrvvtZX2qrd8CFtd2rLRQn4PYNi" alt="drawing" style="width:500px;"/><br>

<img src="https://drive.google.com/uc?id=1cNwp5-gcLDnWtJQP2eyJXXQt4EnjzRXP" alt="drawing" style="width:500px;"/><br>

<img src="https://drive.google.com/uc?id=1hOuZNQAESD9ZSolSGwGbaKFKFg2LiPlS" alt="drawing" style="width:500px;"/>

##### Some Additional resources to know more about these two concepts

[Medium Article 1](https://medium.com/@tomhcorbin/repartitioning-in-spark-repartition-vs-coalesce-5e2fde5fa471#:~:text=The%20repartition%20method%20in%20Spark,partitions%20(which%20is%20200))

[Medium Article 2](https://medium.com/@deepa.account/spark-coalesce-and-repartition-c61cc027ff64)


###### Additional Note:

[What is default Parallelism parameter?](https://subhamkharwal.medium.com/pyspark-the-factor-of-cores-e884b2d5af6c#:~:text=Default%20Parallelism%20is%20a%20very%20standout%20factor%20in%20Spark%20executions,cores%20we%20have%20for%20execution.)

In Spark, default parallelism is the total number of cores available for execution. The default value for this configuration<br>
is the number of cores on all nodes in a cluster. On local, it is set to the number of cores on your system.

[Go to this link to know something more on default Parallelism](https://kontext.tech/article/1149/differences-between-sparksqlshufflepartitions-and-sparkdefaultparallelism)

[What is maxPartitionBytes Parameter](https://kontext.tech/article/1150/about-spark-configuration-sparksqlfilesmaxpatitionbytes#:~:text=Spark%20configuration%20property%20spark.,%2C%20ORC%2C%20CSV%2C%20etc.)

* In Apache Spark, the maxPartitionBytes parameter is a configurable setting that determines the maximum size of each partition in bytes.<br>
It specifies the upper limit to control the partition size and avoid excessively large or small partitions.
* The default value is set to 128 MB (134217728 bytes).

##### Some common resources for all the above topics

[Linked In article](https://www.linkedin.com/pulse/understanding-partitioning-spark-3-levels-taral-desai/)

[Medium Article](https://oindrila-chakraborty88.medium.com/introduction-to-partition-in-apache-spark-66e005c6e15d)

[ProjectPro](https://www.projectpro.io/article/how-data-partitioning-in-spark-helps-achieve-more-parallelism/297#:~:text=In%20Spark%2C%20one%20should%20carefully,with%20the%20number%20of%20partitions.)

##### Some bonus links from Google bard on personal doubts

[Question 1](https://g.co/bard/share/5e56c8770f46)

[Question 2-4](https://g.co/bard/share/8e399bbc4b4f)



In [ ]:
# Reading the flight data 

flight_df = spark.read.format("csv")\
               .option("header","true")\
               .option("inferSchema","true")\
               .option("mode","FAILFAST")\
               .load("/FileStore/tables/flight_data.csv")

In [ ]:
# Displaying the flight_df dataframe
flight_df.show(truncate=False)

+------------------------+-------------------+-----+
|DEST_COUNTRY_NAME       |ORIGIN_COUNTRY_NAME|count|
+------------------------+-------------------+-----+
|United States           |Romania            |15   |
|United States           |Croatia            |1    |
|United States           |Ireland            |344  |
|Egypt                   |United States      |15   |
|United States           |India              |62   |
|United States           |Singapore          |1    |
|United States           |Grenada            |62   |
|Costa Rica              |United States      |588  |
|Senegal                 |United States      |40   |
|Moldova                 |United States      |1    |
|United States           |Sint Maarten       |325  |
|United States           |Marshall Islands   |39   |
|Guyana                  |United States      |64   |
|Malta                   |United States      |1    |
|Anguilla                |United States      |41   |
|Bolivia                 |United States      |

In [ ]:
# Count the number of record in dataframe
flight_df.count()

Out[8]: 256

In [ ]:
# To know the number of partitions we have to use getNumPartitions function which returns the number of partitions in RDD.
# Since we can't apply getNumPartitions directly on dataframe so at first we need to convert it to RDD which shown below.

flight_df.rdd.getNumPartitions()

# To know exactly why it gave 1 partition refer to above link named "Question 2-4"

Out[4]: 1

In [ ]:
# Repartitioning the flight_df with 4 partitions
partitioned_flight_df = flight_df.repartition(4)

In [ ]:
# Showing the number of records with it's corresponding partitionId
from pyspark.sql.functions import *

partitioned_flight_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   64|
|          1|   64|
|          2|   64|
|          3|   64|
+-----------+-----+



In [ ]:
# Using repartition() method you can also do the partition by single column name, or multiple columns.
# Syntax : df.repartition(num_partitions, column)
partitioned_on_column = flight_df.repartition(300,"ORIGIN_COUNTRY_NAME")

In [ ]:
# Checking the total partition of newly created dataframe after repartition by column
partitioned_on_column.rdd.getNumPartitions

Out[15]: <bound method RDD.getNumPartitions of MapPartitionsRDD[68] at javaToPython at NativeMethodAccessorImpl.java:0>

In [ ]:
# Checking the total number of records per partitions.
# When the total number of records falls below the specified partition count in Spark, the framework compensates
# by placing null values in the additional partitions.

partitioned_on_column.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|    1|
|          2|    2|
|          7|    1|
|         10|    1|
|         13|    1|
|         15|    2|
|         16|    2|
|         19|    2|
|         22|    1|
|         28|    1|
|         31|    1|
|         39|    1|
|         43|    1|
|         44|    1|
|         45|    2|
|         48|    1|
|         53|    1|
|         55|    1|
|         65|    1|
|         70|    1|
+-----------+-----+
only showing top 20 rows



In [ ]:
# Creating a new dataframe with 8 partitions to demonstrate the coalesce function
coalesce_flight_df = flight_df.repartition(8)

In [ ]:
# The partition will be evenely distributed as we have used the repartition
coalesce_flight_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   32|
|          1|   32|
|          2|   32|
|          3|   32|
|          4|   32|
|          5|   32|
|          6|   32|
|          7|   32|
+-----------+-----+



In [ ]:
# So we have 8 a dataframe with 8 partitions now let's redistribute the data using coalesce
# Let's say we need 3 partitions as we can only descrease the number of partition in coalesce
# Note : coalesce only takes one argument i.e number of partition we want

three_coalesce_flight_df = coalesce_flight_df.coalesce(3)

In [ ]:
# As we know coalesce will unevenenly distribute the data
three_coalesce_flight_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   64|
|          1|   96|
|          2|   96|
+-----------+-----+



In [ ]:
# When utilizing repartitioning with the same partition count (e.g., 3), Spark ensures an even distribution of records
# across the partitions, in contrast to the specific behavior observed when using coalesce. 

# Consider the following example to witness the uniform distribution achieved through repartitioning.
three_repartitioned_flight_df = coalesce_flight_df.repartition(3)

# Display the total number of records corresponding to it's partitionId
three_repartitioned_flight_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()


+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   85|
|          1|   86|
|          2|   85|
+-----------+-----+



In [ ]:
# Let's see what happens when we try to increase the number of partition using coalesce.
# I have used the coalesce_flight_df which has 8 partitions.
coalesce_flight_df.coalesce(10).rdd.getNumPartitions()
# As you can see we can't increase the total number of partition in coalesce.

Out[25]: 8